## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Torbay,CA,47.67,-52.73,48.20,57,20,5.82,few clouds
1,1,Mataura,NZ,-46.19,168.86,45.00,73,78,10.00,light rain
2,2,Kapaa,US,22.08,-159.32,84.20,70,1,13.87,clear sky
3,3,Aykhal,RU,66.00,111.50,14.00,92,20,2.24,few clouds
4,4,Zonguldak,TR,41.25,31.83,61.77,90,39,4.23,scattered clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Kapaa,US,22.08,-159.32,84.20,70,1,13.87,clear sky
10,10,Progreso,MX,21.28,-89.67,90.00,70,75,4.70,broken clouds
14,14,Nhulunbuy,AU,-12.23,136.77,77.00,88,60,8.05,broken clouds
15,15,Agadez,NE,19.75,10.25,80.33,17,0,1.23,clear sky
16,16,Saint George,US,37.10,-113.58,82.99,13,1,3.51,clear sky
20,20,San Policarpo,PH,12.18,125.51,76.64,93,100,11.52,heavy intensity rain
29,29,Hilo,US,19.73,-155.09,82.40,65,75,4.70,broken clouds
30,30,Labuhan,ID,-6.88,112.21,78.40,82,100,7.23,overcast clouds
32,32,Atuona,PF,-9.80,-139.03,78.17,74,7,19.06,clear sky
34,34,Puerto Ayora,EC,-0.74,-90.35,78.80,65,20,16.11,few clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                217
City                   217
Country                216
Lat                    217
Lng                    217
Max Temp               217
Humidity               217
Cloudiness             217
Wind Speed             217
Current Description    217
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kapaa,US,84.20,clear sky,22.08,-159.32,
10,Progreso,MX,90.00,broken clouds,21.28,-89.67,
14,Nhulunbuy,AU,77.00,broken clouds,-12.23,136.77,
15,Agadez,NE,80.33,clear sky,19.75,10.25,
16,Saint George,US,82.99,clear sky,37.10,-113.58,
20,San Policarpo,PH,76.64,heavy intensity rain,12.18,125.51,
29,Hilo,US,82.40,broken clouds,19.73,-155.09,
30,Labuhan,ID,78.40,overcast clouds,-6.88,112.21,
32,Atuona,PF,78.17,clear sky,-9.80,-139.03,
34,Puerto Ayora,EC,78.80,few clouds,-0.74,-90.35,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    response = requests.get(base_url, params=params)
    #print(response.url)
    hotels = response.json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping")

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace("", np.nan, inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kapaa,US,84.20,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
10,Progreso,MX,90.00,broken clouds,21.28,-89.67,Playa Linda Hotel
14,Nhulunbuy,AU,77.00,broken clouds,-12.23,136.77,East Arnhem Real Estate
15,Agadez,NE,80.33,clear sky,19.75,10.25,NaN
16,Saint George,US,82.99,clear sky,37.10,-113.58,Best Western Plus Abbey Inn
...,...,...,...,...,...,...,...
670,Maceio,BR,78.80,few clouds,-9.67,-35.74,Mercure Maceio Pajucara Hotel
671,Mecca,SA,76.50,clear sky,21.43,39.83,Hotel Fairmont Makkah Clock Royal Tower
678,Boulder,US,75.20,clear sky,40.08,-105.35,Deck House Boulder
680,Sumenep,ID,80.46,overcast clouds,-7.02,113.87,OYO 3271 Homestay Pandian


In [11]:
clean_hotel_df = hotel_df.dropna()

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))